In [19]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from tensorflow import keras
import tensorflow as tf
from keras_preprocessing import image as op

    


In [ ]:

def predict(image):
  def rotateImage(im, angle):
   (h, w) = im.shape[:2]
   center = (w / 2, h / 2)
   M = cv2.getRotationMatrix2D(center,angle,1.0)
   rotated_image = cv2.warpAffine(im, M, (w,h))
   return rotated_image
  def shadow_remove(img):
      rgb_planes = cv2.split(img)
      result_norm_planes = []
      for plane in rgb_planes:
          dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
          bg_img = cv2.medianBlur(dilated_img, 21)
          diff_img = 255 - cv2.absdiff(plane, bg_img)
          norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
          result_norm_planes.append(norm_img)
      shadowremov = cv2.merge(result_norm_planes)
      return shadowremov

  
  def threshing(ohh):
    img = cv2.cvtColor(ohh, cv2.COLOR_BGR2GRAY)

    ret,thresh1 = cv2.threshold(img,200,255,cv2.THRESH_BINARY)
    thresh1 = cv2.bitwise_not(thresh1)
    kernel = np.ones((3,3),np.uint8)
    kernel2 = np.ones((3,3),np.uint8)
    dilation = cv2.dilate(thresh1,kernel,iterations =1)

    opening = cv2.erode(dilation,kernel2,iterations =1)
    return opening
  mapping=["क","ठ", "ड", "ढ", "त", "ख", "प","फ","म", "य", "र", "ल","व","ष", "ह","क्ष", "त्र","ज्ञ","घ", "च", "ज"]
  shad = shadow_remove(image)
  img = cv2.cvtColor(shad, cv2.COLOR_BGR2GRAY)
  ret,thresh1 = cv2.threshold(img,200,255,cv2.THRESH_BINARY)
  thresh1 = cv2.bitwise_not(thresh1)
  kernel = np.ones((6,6),np.uint8)
  kernel2 = np.ones((8,8),np.uint8)
  dilation = cv2.dilate(thresh1,kernel,iterations =2)
  opening = cv2.erode(dilation,kernel2,iterations =1)
  box=np.empty(4, dtype=int)
  box[0]=0
  box[1]=0
  box[2]=100000
  box[3]=100000

  img_area=img.shape[0]*img.shape[1]
  contours,_=cv2.findContours(opening,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


  rect=[]
  mids=[]
  rd=[]
  for cnt in contours:
      if cv2.contourArea(cnt)>img_area*0.9:
          continue
      if cv2.contourArea(cnt)>img_area/600:
          x,y,w,h=cv2.boundingRect(cnt)
          if x<box[2]:
            box[2]=x
          if y<box[3]:
            box[3]=y
          if x+w>box[0]:
            box[0]=x+w
          if y+h>box[1]:
            box[1]=y+h

  image_dash=opening[max(box[3]-10,0):min(box[1]+10,int(opening.shape[0])),max(box[2]-10,0):min(int(opening.shape[1]),box[0]+10)]
  image_dash_dash=image[max(box[3]-10,0):min(box[1]+10,int(image.shape[0])),max(box[2]-10,0):min(int(image.shape[1]),box[0]+10)]
  contourss,_=cv2.findContours(image_dash,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  for cnt in contourss:
    if cv2.contourArea(contourss[0])<cv2.contourArea(cnt):
      contourss[0]=cnt
  rect = cv2.minAreaRect(contourss[0])
  boxx = cv2.boxPoints(rect)
  boxx = np.int0(boxx)
  im=image_dash_dash.copy()
  im = cv2.drawContours(im,[boxx],0,(0,0,255),2)
  x=rect[2]
  if x<-45:
    x=(90+rect[2])
  rot_image=rotateImage(im,x)
  rot_image_dash=rotateImage(image_dash,x)
  rot_image_dash_dash=rotateImage(image_dash_dash,x)
  box=np.empty(4, dtype=int)
  box[0]=0
  box[1]=0
  box[2]=100000
  box[3]=100000

  img_area=rot_image_dash.shape[0]*rot_image_dash.shape[1]
  contours,_=cv2.findContours(rot_image_dash.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


  rect=[]
  mids=[]
  rd=[]
  for cnt in contours:
      if cv2.contourArea(cnt)>img_area*0.9:
          continue
      if cv2.contourArea(cnt)>img_area/600:
          x,y,w,h=cv2.boundingRect(cnt)
          if x<box[2]:
            box[2]=x
          if y<box[3]:
            box[3]=y
          if x+w>box[0]:
            box[0]=x+w
          if y+h>box[1]:
            box[1]=y+h
  final_image=rot_image_dash[box[3]:box[1],box[2]:box[0]]
  final_image_dash=rot_image_dash_dash[box[3]:box[1],box[2]:box[0]]
  finalImage=threshing(shadow_remove(final_image_dash))
  img4=final_image.copy()
  y=int(img4.shape[0])
  x=int(img4.shape[1])
  img4 = cv2.rectangle(img4, (0,0), (x,int(0.23*y)), (0,0,0), -1)

  new_img=img4.copy()
  img_area=new_img.shape[0]*new_img.shape[1]
  contours,_=cv2.findContours(new_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  rect=[]
  mids=[]
  rd=[]
  for cnt in contours:
      if cv2.contourArea(cnt)>img_area/2:
          continue
      if cv2.contourArea(cnt)>img_area/300:
          x,y,w,h=cv2.boundingRect(cnt)
          rect.append([x,y,w,h])
          mids.append([x+w/2,y+h/2])


  img2=new_img.copy()
  for k in rect:
      i=[k[0]+k[2]/2, k[1]+k[3]/2]
      check=0
      for j in rd:
          if i[0]>j[0] and i[0]<(j[0]+j[2]) and i[1]>j[1] and i[1]<(j[1]+j[3]):
              check=-1
              break;
      if check==0 and k[3]<2*k[2]:
          rd.append([k[0],k[1],k[2],k[3]])
  rd.sort()
  lis=[]
  xx=int(final_image_dash.shape[0])
  yy=int(final_image_dash.shape[1])
  for x,y,w,h in rd:
    pad =  threshing(shadow_remove(final_image_dash[0:min(yy,y+h+2),max(0,x-2):min(x+w+2,yy)]))
    lis.append(pad)
  liss=lis.copy()
  k=len(lis)
  for i in range(k):
    lis[i]=cv2.resize(lis[i],(32,32))
    x = op.img_to_array(lis[i]) 
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    lis[i]=x
  model = keras.models.load_model('model21.h5')
  y_pred=[]
  for i in range(k):
    y_pred.append(np.argmax(model.predict(lis[i]), axis=-1))
  t=[]
  for i in y_pred:
    t.append(mapping[int(i)])
  return t

In [ ]:

def test():
    '''
    We will be using a similar template to test your code
    '''
    image_paths = ['./image1','./image2',',./imagen']
    correct_answers = [list1,list2,listn]
    score = 0
    multiplication_factor=2 #depends on character set size

    for i,image_path in enumerate(image_paths):
        image = cv2.imread(image_path) # This input format wont change
        answer = predict(image) # a list is expected
        print(''.join(answer))# will be the output string

        n=0
        for j in range(len(answer)):
            if correct_answers[i][j] == answer[j]:
                n+=1
                
        if(n==len(correct_answers[i])):
            score += len(correct_answers[i])*multiplication_factor

        else:
            score += n*2
        
    
    print('The final score of the participant is',score)

